In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd "/content/drive/My Drive/Colab Notebooks/Matrix"

/content/drive/My Drive/Colab Notebooks/Matrix


In [7]:
ls

data/  HelloGithub.ipynb  matrix_one/  README.md


In [0]:
df = pd.read_csv('data/men_shoes.csv',low_memory=False)

In [10]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [13]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [17]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [16]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [18]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [19]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [21]:
test = {'key': 'value'}
test['key']

'value'

In [22]:
#
str(test)

"{'key': 'value'}"

In [24]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
    
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [26]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [27]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [31]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [32]:
df[ df['feat_athlete'].isnull() ].shape

(18272, 1002)

In [33]:
df.shape[0]

18280

In [36]:
df[ False == df['feat_athlete'].isnull() ].shape[0]

8

In [37]:
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [47]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
#2
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
#3
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [50]:
df[ df.brand == df.feat_brand ].shape

(52, 1007)

In [52]:
df[ df.brand == df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [54]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [56]:
df[ df.brand == df.feat_brand ].shape

(8846, 1007)

In [57]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.27043685524772, 4.268708186162379)

In [0]:
feats_Add = [ 'feat_navy blue_cat',
 'feat_style number_cat',
 'feat_bridge width_cat',
 'feat_hands_cat',
 'feat_weather-resistant_cat',
 'feat_date first available at amazon.ca_cat',
 'feat_fabric material_cat',
 'feat_frame depth_cat',
 'feat_motion control_cat',
 'feat_thick, warm, cozy_cat',
 'feat_sizearm_cat',
 'feat_resolution_cat',
 'feat_product #_cat',
 'feat_date first available at amazon.co.uk_cat',
 'feat_height_cat',
 'feat_contains batteries_cat',
 'feat_fits most screen size_cat',
 'feat_diameter_cat',
 'feat_flame resistant_cat',
 'feat_closure style_cat',
 'feat_auto shut-off_cat',
 'feat_ean_cat',
 'feat_fabric type_cat',
 'feat_condition_cat',
 'feat_removable hood_cat',
 'feat_display technology_cat',
 'feat_sku_cat',
 'feat_country of origin assembly:_cat',
 'feat_reinforced knee_cat',
 'feat_age range_cat',
 'feat_type of sole_cat',
 'feat_shipping to usa_cat',
 'feat_manufacturer number_cat',
 'feat_parts_cat',
 'feat_msrp_cat',
 'feat_country//region of manufacture_cat',
 'feat_special features_cat',
 'feat_vendor description_cat',
 'feat_charger included_cat',
 'feat_primary shelf id_cat',
 'feat_profession_cat',
 'feat_dimensions_cat',
 'feat_item width_cat',
 'feat_ring style_cat',
 'feat_heat zones_cat',
 'feat_material detail-1_cat',
 'feat_battery voltage_cat',
 'feat_removable liner_cat',
 'feat_year made_cat',
 'feat_animal type_cat',
 'feat_label_cat',
 'feat_hairstyle_cat',
 'feat_leather :_cat',
 'feat_fuel type_cat',
 'feat_product id_cat',
 'feat_item spec attr - style (ebay)_cat',
 'feat_frame color_cat',
 'feat_color/finish family_cat',
 'feat_front style_cat',
 'feat_antiscratch lens coating_cat',
 'feat_model no._cat',
 'feat_elastic banded waist_cat',
 'feat_is waterproof_cat',
 'feat_water-resistant_cat',
 'feat_black_cat',
 'feat_sizebridge_cat',
 'feat_nickel free_cat',
 'feat_display_cat',
 'feat_fits brands_cat',
 'feat_age end_cat',
 'feat_is water-resistant_cat',
 'feat_athlete_cat',
 'feat_stability_cat',
 'feat_safety apparel product type_cat',
 'feat_free item_cat',
 'feat_country/ organization_cat',
 'feat_lens_cat',
 'feat_light transmission (vlt)_cat',
 'feat_shoe width_cat',
 'feat_is weather-resistant_cat',
 'feat_rise_cat',
 'feat_country/region of manufacture_cat',
 'feat_bridge size:_cat',
 'feat_case back_cat',
 'feat_fine or fashion_cat',
 'feat_era_cat',
 'feat_wheeled_cat',
 'feat_shipping weight_cat',
 'feat_conflict_cat',
 'feat_character_cat',
 'feat_hammer loop_cat',
 'feat_manufacturer_cat',
 'feat_seller inventory number_cat',
 'feat_atpv arc rating_cat',
 'feat_has expiration_cat',
 'feat_mpn#_cat',
 'feat_front_cat',
 'feat_chronograph_cat',
 'feat_target audience_cat',
 'feat_shoe closure_cat',
 'feat_face shape_cat',
 'feat_pocket_cat',
 'feat_sleeve style_cat',
 'feat_casing_cat',
 'feat_accessory type_cat',
 'feat_assembled product weight_cat',
 'feat_frame style_cat',
 'feat_mechanic_cat',
 'feat_euro size_cat',
 'feat_case finish_cat',
 'feat_global composite sports type_cat',
 'feat_leg_cat',
 'feat_watch power source_cat',
 'feat_100% adidas authentic_cat',
 'feat_item color_cat',
 'feat_mirrored_cat',
 'feat_auto_cat',
 'feat_safety feature_cat',
 'feat_arm_cat',
 'feat_theme_cat',
 'feat_nike product #_cat',
 'feat_age start_cat',
 'feat_is lined_cat',
 'feat_fit:_cat',
 'feat_nike id_cat',
 'feat_lined_cat',
 'feat_materials_cat',
 'feat_country of manufacture_cat',
 'feat_material composition_cat',
 'feat_item style_cat',
 'feat_band color_cat',
 'feat_domestic shipping_cat',
 'feat_certifications and listings_cat',
 'feat_gold karat_cat',
 'feat_golf shoe type_cat',
 'feat_measurements:_cat',
 'feat_lens technology_cat',
 'feat_multi pack indicator_cat',
 'feat_sku number_cat',
 'feat_series name_cat',
 'feat_protection_cat',
 'feat_weight_cat',
 'feat_is recyclable_cat',
 'feat_location - city/state_cat',
 'feat_shoe category_cat',
 'feat_frame material_cat',
 'feat_location - country_cat',
 'feat_jacket length_cat',
 'feat_general warranty_cat',
 'feat_age segment_cat',
 'feat_case type:_cat',
 'feat_compatible devices_cat',
 'feat_lens color family_cat',
 'feat_материал_cat',
 'feat_stock no._cat',
 'feat_alarm_cat',
 'feat_expandable_cat',
 'feat_boot height_cat',
 'feat_product name_cat',
 'feat_cushioning_cat',
 'feat_is dark sky-compliant_cat',
 'feat_fit_cat',
 'feat_diamond color_cat',
 'feat_waist size_cat',
 'feat_polarized lenses_cat',
 'feat_has mercury_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_combined shipping_cat',
 'feat_isbn_cat',
 'feat_bed size_cat',
 'feat_lens tint_cat',
 'feat_colour_cat',
 'feat_band manufacturer_cat',
 'feat_pant style_cat',
 'feat_adidas_cat',
 'feat_movement_cat',
 'feat_cleaning, care & maintenance_cat',
 'feat_clothing size_cat',
 'feat_heel height_cat',
 'feat_item type_cat',
 'feat_main colour_cat',
 'feat_picture_cat',
 'feat_age group_cat',
 'feat_country////region of manufacture_cat',
 'feat_tactical_cat',
 'feat_power type_cat',
 'feat_case type_cat',
 'feat_very popular bag now a days_cat',
 'feat_szie_cat',
 'feat_fits models_cat',
 'feat_heart rate monitor_cat',
 'feat_digital camera_cat',
 'feat_transactionid_cat',
 'feat_recommended location_cat',
 'feat_wheel type_cat',
 'feat_clasp type_cat',
 'feat_crown_cat',
 'feat_variant group id_cat',
 'feat_licensed_cat',
 'feat_spikes type_cat',
 'feat_closure_cat',
 'feat_adjustable_cat',
 'feat_international shipping?_cat',
 'feat_ring size_cat',
 'feat_inv_cat',
 'feat_model_cat',
 'feat_authenticity_cat',
 'feat_main color_cat',
 'feat_dial markers_cat',
 'feat_clothing category_cat',
 'feat_is polarized_cat',
 'feat_product dimensions_cat',
 'feat_interior pockets_cat',
 'feat_number of batteries included_cat',
 'feat_what includes_cat',
 'feat_batteries required_cat',
 'feat_protects against_cat',
 'feat_hat style_cat',
 'feat_shade_cat',
 'feat_fabric content_cat',
 'feat_inseam_cat',
 'feat_country of origin - assembly_cat',
 'feat_sub type_cat',
 'feat_case tone_cat',
 'feat_hood_cat',
 'feat_country of origin components:_cat',
 'feat_manufacturer_part_number_cat',
 'feat_place of origin_cat',
 'feat_sock size_cat',
 'feat_hardsided or softsided_cat',
 'feat_polarized?_cat',
 'feat_best sellers rank_cat',
 'feat_bridge/temple size:_cat',
 'feat_chain included_cat',
 'feat_labor_cat',
 'feat_characteristics_cat',
 'feat_assembled product dimensions (l x w x h)_cat',
 'feat_metal color_cat',
 'feat_article_cat',
 'feat_designed for_cat',
 'feat_style #_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_fits model_cat',
 'feat_eyewear frame style_cat',
 'feat_work shoes_cat',
 'feat_type_cat',
 'feat_country of origin - components_cat',
 'feat_has paper wood_cat',
 'feat_airport friendly_cat',
 'feat_chest size_cat',
 'feat_foot arch_cat',
 'feat_casual & dress shoe style_cat',
 'feat_waterproof_cat',
 'feat_gps_cat',
 'feat_use_cat',
 'feat_hunting_cat',
 'feat_style:_cat',
 'feat_clothing size type_cat',
 'feat_footwear type_cat',
 'feat_leather grade_cat',
 'feat_lens socket width_cat',
 'feat_purpose_cat',
 'feat_overall material_cat',
 'feat_package_cat',
 'feat_resizable_cat',
 'feat_date_cat',
 'feat_frame type_cat',
 'feat_watch case shape_cat',
 'feat_guaranteed authentic_cat',
 'feat_recommended use_cat',
 'feat_sports team_cat',
 'feat_terrain_cat',
 'feat_quantity in set_cat',
 'feat_video game platform_cat',
 'feat_style name_cat',
 'feat_is energy star-certified_cat',
 'feat_hooded_cat',
 'feat_part type_cat',
 'feat_sports league_cat',
 'feat_stylenumber_cat',
 'feat_features_cat',
 'feat_smart watch_cat',
 'feat_lining_cat',
 'feat_country of origin_cat',
 'feat_catalog_cat',
 'feat_number of pockets_cat',
 'feat_bag size range_cat',
 'feat_capacity_cat',
 'feat_polarized_cat',
 'feat_age gender group_cat',
 'feat_width_cat',
 'feat_resolution megapixels_cat',
 'feat_walmart no._cat',
 'feat_finish_cat',
 'feat_shoe fastening type_cat',
 'feat_wash_cat',
 'feat_shirt size_cat',
 'feat_uv rating_cat',
 'feat_subdial_cat',
 'feat_clothing type_cat',
 'feat_wind resistant_cat',
 'feat_lens type_cat',
 'feat_fabrication_cat',
 'feat_waist_cat',
 'feat_includes_cat',
 'feat_contains cubic zirconia_cat',
 'feat_designer_cat',
 'feat_lens width_cat',
 'feat_watch style_cat',
 'feat_fabric care instructions_cat',
 'feat_band type_cat',
 'feat_number of items_cat',
 'feat_case diameter_cat',
 'feat_length_cat',
 'feat_band length_cat',
 'feat_eye size:_cat',
 'feat_shape_cat',
 'feat_contained battery type_cat',
 'feat_toe type_cat',
 'feat_battery type_cat',
 'feat_fishing_cat',
 'feat_stone type_cat',
 'feat_sub style_cat',
 'feat_enhanced visibility (reflective)_cat',
 'feat_to fit_cat',
 'feat_temple length_cat',
 'feat_primary color_cat',
 'feat_number of pieces_cat',
 'feat_audience_cat',
 'feat_fabric_cat',
 'feat_sleeve length_cat',
 'feat_part number_cat',
 'feat_year_cat',
 'feat_design_cat',
 'feat_name_cat',
 'feat_assembled in country of origin_cat',
 'feat_military style_cat',
 'feat_outer material_cat']

In [80]:
feats = ['brand_cat','feat_color_cat','feat_gender_cat','feat_manufacturer part number_cat','feat_material_cat']
#feats += feats_Add

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.81199446334024, 4.3140850085815705)

In [81]:
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  m = RandomForestRegressor(max_depth=5,n_estimators=100, random_state=0)
  m.fit(X, y)

  perm = PermutationImportance(m, random_state=1).fit(X, y);
  eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2429 ± 0.0099,brand_cat
0.1012 ± 0.0072,feat_material_cat
0.0121 ± 0.0009,feat_weight_cat
0.0091 ± 0.0008,feat_adjustable_cat
0.0079 ± 0.0009,feat_resizable_cat
0.0065 ± 0.0009,feat_shoe category_cat
0.0057 ± 0.0015,feat_color_cat
0.0055 ± 0.0003,feat_fabric content_cat
0.0045 ± 0.0006,feat_case diameter_cat
0.0043 ± 0.0002,feat_fabric material_cat


In [63]:
feats = ['brand_cat','feat_color_cat','feat_gender_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.85339203611592, 4.704881723420811)

In [64]:
df['brand'].value_counts(normalize=True)

nike               0.097210
puma               0.033315
ralph lauren       0.028775
vans               0.021116
new balance        0.020295
                     ...   
trooper america    0.000055
zanzara            0.000055
hulkamania         0.000055
c1rca              0.000055
pony               0.000055
Name: brand, Length: 1732, dtype: float64

In [67]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [68]:
feats = ['brand_cat','feat_color_cat','feat_gender_cat','feat_style_cat','feat_sport_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.83938523649181, 4.955675441698302)

In [70]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{},
       {'sport': 'soccer', 'style': 'cleats', 'country/region of manufacture': 'bosnia and herzegovina', 'shade': 'white-total crimson-vivid purple', 'type': 'cleats', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': "men's shoes", 'features': 'adjustable cleats ,�� comfortable material', 'brand': 'nike'},
       {'material': 'synthetic', 'gender': 'men', 'size': '9', 'shoe category': "men's shoes", 'color': 'white/anthracite/pure platinum/wolf grey', 'model': 'nikm-flextrainer6w-whiteant455.9m', 'manufacturer part number': 'nikm-flextrainer6w-whiteant455.9m', 'brand': 'nike', 'age group': 'adult'},
       {'sport': 'casual shoes', 'occasion': 'casual shoes', 'material': 'mesh', 'gender': 'men', 'shoe size': '12 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'mesh', 'color': 'black/white/anthracite', 'model': 'nikm-zoomwinflow2-blackwhi240.12m', 'shoe closure': 'lace-

In [71]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
infant                 4
mens                   4
boys'                  3
women ,�� unisex       2
men||women             2
youth                  2
women                  2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [73]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'featcertifications and listings',
 'featmulti pack indicator',
 'featlocation - city/state',
 'featshoe category',
 'featlocation - country',
 'featrecommended location',
 'featclothing category',
 'featcatalog',
 'featfabrication',
 'feat_certifications and listings',
 'feat_multi pack indicator',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_location - country',
 'feat_recommended location',
 'feat_clothing category',
 'feat_catalog',
 'feat_fabrication',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_color mapping_cat',
 'feat_sock style_cat',
 'feat_size_cat',
 'feat_country of manufacturer_cat',
 'feat_uv protection_cat',
 'feat_color family_cat',
 'feat_dial material_cat',
 'feat_weather resistant_cat',
 'feat_size/dimensions_cat',
 'feat_lens material:_cat',
 'feat_charge time (minutes)_cat',
 'feat_occasion_cat',
 'fea

In [82]:
!git add day5.ipynb

fatal: pathspec 'day5.ipynb' did not match any files


In [83]:
ls

data/  HelloGithub.ipynb  matrix_one/  README.md
